In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import os
import datetime
from sklearn.pipeline import Pipeline
from dateutil.relativedelta import relativedelta
import warnings

In [4]:
df = pd.read_csv('d:/code/data/data.csv', encoding='ISO-8859-1')

In [5]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [6]:
# Suppress the SettingWithCopyWarning
pd.set_option('mode.chained_assignment', None)
pd.set_option('future.no_silent_downcasting', True)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [75]:
df.InvoiceDate.min(), df.InvoiceDate.max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

# Function for one month duration snapshot

In [33]:
# Function to get the snapshot of the data at a given time in the past, the time_shot of the snapshot is one month
## Input
##      df: the original data
##      time_snapshot: the time of the snapshot
##      time_shot: the time_shot of the snapshot (1 month ago, 2 months ago, 3 months ago, etc.)
## Output
##      df_snapshot: the snapshot of the data at the given time in the past. The duration of the snapshot is one month
##                    The snapshot includes the total amount of successful orders and the number of successful orders for each customer 

def snap_shot_month(df: pd.DataFrame, time_snapshot: datetime.datetime, time_shot: int = 1) -> pd.DataFrame:

    # Get list of unique customers until the time_snapshot
    customers = df[df['InvoiceDate'] < time_snapshot]['CustomerID'].unique()
    
    # Filter the data to get the snapshot of one month
    df_filter = df[(df['InvoiceDate'] >= time_snapshot - relativedelta(months=time_shot)) & (df['InvoiceDate'] <= time_snapshot- relativedelta(months=time_shot-1))]

    # Create a new column to store the total amount of each transaction
    df_filter['total_amount'] = df_filter['Quantity'] * df_filter['UnitPrice']  

    # Create a new Dataframe to store the snapshot of the data  
    df_snapshot = pd.DataFrame(columns=['customer_id', 'total_successful_amount_past_1_month', 'num_successful_orders_past_1_month'])

    df_filter_agg = df_filter.groupby(['CustomerID']).agg({'total_amount': 'sum', 'InvoiceDate': 'count'}).reset_index()
    df_filter_agg.rename(columns={'CustomerID':'customer_id', 'InvoiceDate': f'num_successful_orders_past_{time_shot}_month', 'total_amount': f'total_successful_amount_past_{time_shot}_month'}, inplace=True)
    df_snapshot = pd.concat([df_snapshot, df_filter_agg], ignore_index=True) if df_snapshot.shape[0] > 0 else df_filter_agg
 
    
    # Add the time_snapshot to the snapshot dataframe
    df_snapshot['time_snapshot'] = time_snapshot
    
    # Drop the rows with missing values (customer_id)
    df_snapshot.dropna(inplace=True)
    return df_snapshot

In [76]:
# Test the function
time_snapshot = datetime.datetime(2010, 12, 9)
df_snap = snap_shot_month(df, time_snapshot)
df_snap

,customer_id,total_successful_amount_past_1_month,num_successful_orders_past_1_month,time_snapshot
0,12347.0,711.79,31,2010-12-09
1,12386.0,258.90,8,2010-12-09
2,12395.0,346.10,12,2010-12-09
3,12427.0,303.50,10,2010-12-09
4,12431.0,358.25,14,2010-12-09
...,...,...,...,...
534,18225.0,-31.80,1,2010-12-09
535,18229.0,653.20,10,2010-12-09
536,18239.0,438.10,29,2010-12-09
537,18259.0,376.30,7,2010-12-09


In [35]:
# Function to get the first days of each month between two dates
def get_first_days_of_months(start_date, end_date):
    # List to store the first days of each month
    first_days = []
    
    # Start from the first day of the start date's month
    current_date = start_date.replace(day=1)
    
    # Loop until the current_date is past the end_date
    while current_date <= end_date:
        first_days.append(current_date)
        current_date += relativedelta(months=1)  # Add one month
        
    return first_days

In [36]:
# Function to get all snapshots of the data at a given time in the past

def snap_shot_all(df: pd.DataFrame,  time_shot: int = 1) -> pd.DataFrame:
    
    # Get time snapshots
    time_snapshots = df['InvoiceDate'].unique()
    time_min = time_snapshots.min().date()
    time_max = time_snapshots.max().date()

    # Create the beginning and ending time of the snapshots
    time_begin = pd.to_datetime(time_min.replace(day=1) + relativedelta(months=time_shot))
    time_end = pd.to_datetime(time_max.replace(day=1)+ relativedelta(months=time_shot))

    # Get the first days of each month between the beginning and ending time 
    first_days = get_first_days_of_months(time_begin, time_end)

    # Create a new Dataframe to store the snapshot of the data  
    df_snapshot_all = pd.DataFrame(columns=['customer_id', 'total_successful_amount_past_1_month', 'num_successful_orders_past_1_month', 'time_snapshot'])

    # Get the snapshot of the data at each time in the past
    for time_snapshot in first_days:
        df_snapshot = snap_shot_month(df, time_snapshot, time_shot=time_shot)
        df_snapshot_all = pd.concat([df_snapshot_all, df_snapshot], ignore_index=True) if df_snapshot_all.shape[0] > 0 else df_snapshot
    
    return df_snapshot_all

In [37]:
snap_shot_all(df)

,customer_id,total_successful_amount_past_1_month,num_successful_orders_past_1_month,time_snapshot
0,12347.0,711.79,31,2011-01-01
1,12348.0,892.80,17,2011-01-01
2,12370.0,1868.02,91,2011-01-01
3,12377.0,1001.52,43,2011-01-01
4,12383.0,600.72,37,2011-01-01
...,...,...,...,...
13670,18245.0,894.25,55,2012-01-01
13671,18272.0,367.88,23,2012-01-01
13672,18273.0,51.00,1,2012-01-01
13673,18282.0,77.84,5,2012-01-01


# Function looking back to the past and looking forward the future

In [38]:
def snapshot_past_month_begin(df: pd.DataFrame, time_snapshot: datetime.datetime, time_shot: int = 1) -> pd.DataFrame:

    # Get list of unique customers until the time_snapshot
    customers_until_time_snapshot = df[df['InvoiceDate'] < time_snapshot]['CustomerID'].dropna().unique()

    # Filter the data to get the snapshot of one month
    df_filter = df[(df['InvoiceDate'] > time_snapshot - relativedelta(months=time_shot)) & (df['InvoiceDate'] <= time_snapshot- relativedelta(months=time_shot-1))]

    # Create a new column to store the total amount of each transaction
    df_filter['total_amount'] = df_filter['Quantity'] * df_filter['UnitPrice']  

    # Create a new Dataframe to store the snapshot of the data  
    df_snapshot = pd.DataFrame(columns=['customer_id', 'total_successful_amount_past_1_month', 'num_successful_orders_past_1_month'])

    # List of all customers in the filtered data
    lst_customer_filter = df_filter['CustomerID'].unique()

    # Get all the customer before the time snapshot-time_shot
    customers_until_past_timeshot = df[(df['InvoiceDate'] < time_snapshot - relativedelta(months=time_shot))]["CustomerID"].unique()
    
  
    # Customers who have successful orders during the time_snapshot-time_shot
    df_snapshot = pd.DataFrame(columns=['customer_id', f'total_successful_amount_past_{time_shot}_month', f'num_successful_orders_past_{time_shot}_month'])
    df_filter_agg = df_filter.groupby('CustomerID').aggregate({'total_amount': 'sum', 'InvoiceDate': 'count'}).reset_index()
    df_filter_agg.rename(columns={'CustomerID':'customer_id', 'InvoiceDate': f'num_successful_orders_past_{time_shot}_month', 'total_amount': f'total_successful_amount_past_{time_shot}_month'}, inplace=True)
    df_snapshot = pd.concat([df_snapshot, df_filter_agg], ignore_index=True) if df_snapshot.shape[0] > 0 else df_filter_agg


    # Customers who have successful orders in the past of the time_snapshot-time_shot but have no successful orders in the filtered data
    df_successful_order = pd.DataFrame(columns=['customer_id', f'total_successful_amount_past_{time_shot}_month', f'num_successful_orders_past_{time_shot}_month'])
    lst_no_orders = [customer_id for customer_id in customers_until_time_snapshot if (customer_id not in lst_customer_filter) & (customer_id in customers_until_past_timeshot)]    
    df_successful_order['customer_id'] = lst_no_orders
    df_successful_order.fillna(0, inplace=True)

    # Customers who have no successful orders in the past of the time_snapshot-time_shot
    df_no_information = pd.DataFrame(columns=['customer_id', f'total_successful_amount_past_{time_shot}_month', f'num_successful_orders_past_{time_shot}_month'])
    lst_no_information = [customer_id for customer_id in customers_until_time_snapshot if (customer_id not in lst_customer_filter) & (customer_id not in customers_until_past_timeshot)]
    df_no_information['customer_id'] = lst_no_information
    

    df_snapshot = pd.concat([df_snapshot, df_successful_order, df_no_information], ignore_index=True) if df_snapshot.shape[0] > 0 else pd.concat([df_successful_order, df_no_information], ignore_index=True)

    # Add the time_snapshot to the snapshot dataframe
    df_snapshot['time_snapshot'] = time_snapshot    

    return df_snapshot

In [84]:
# Test the functionq
time_snapshot = datetime.datetime(2011, 12, 9)
df_past = snapshot_past_month_begin(df, time_snapshot, time_shot=2)
df_past

,customer_id,total_successful_amount_past_2_month,num_successful_orders_past_2_month,time_snapshot
0,12347.0,1294.32,47,2011-12-09
1,12352.0,311.73,15,2011-12-09
2,12357.0,6207.67,131,2011-12-09
3,12359.0,2876.85,105,2011-12-09
4,12360.0,1043.78,54,2011-12-09
...,...,...,...,...
4366,15471.0,NaN,NaN,2011-12-09
4367,13436.0,NaN,NaN,2011-12-09
4368,15520.0,NaN,NaN,2011-12-09
4369,13298.0,NaN,NaN,2011-12-09


In [40]:
def snapshot_future_month_begin(df: pd.DataFrame, time_snapshot: datetime.datetime, time_shot: int = 1) -> pd.DataFrame:

    # Get list of unique customers until the time_snapshot
    customers_until_time_snapshot = df[df['InvoiceDate'] < time_snapshot]['CustomerID'].dropna().unique()    

    # Filter the data to get the snapshot of one month
    df_filter = df[(df['InvoiceDate'] > time_snapshot + relativedelta(months=time_shot-1)) & (df['InvoiceDate'] <= time_snapshot + relativedelta(months=time_shot))]
    df_filter = df_filter[df_filter.CustomerID.isin(customers_until_time_snapshot)]
    
    # Maximal date of the data
    max_date = df['InvoiceDate'].max()
    # Check if snapshot time is in the time frame of data or not
    if time_snapshot + relativedelta(months=time_shot-1) < max_date:

        # Create a new column to store the total amount of each transaction
        df_filter['total_amount'] = df_filter['Quantity'] * df_filter['UnitPrice']  

        # Create a new Dataframe to store the snapshot of the data  
        df_snapshot = pd.DataFrame(columns=['customer_id', f'total_successful_amount_future_{time_shot}_month', f'num_successful_orders_future_{time_shot}_month'])

        # List of all customers in the filtered data
        lst_customer_filter = df_filter['CustomerID'].unique()

    
        df_snapshot = pd.DataFrame(columns=['customer_id', f'total_successful_amount_future_{time_shot}_month', f'num_successful_orders_future_{time_shot}_month'])
        df_filter_agg = df_filter.groupby('CustomerID').aggregate({'total_amount': 'sum', 'InvoiceDate': 'count'}).reset_index()
        df_filter_agg.rename(columns={'CustomerID':'customer_id', 'InvoiceDate': f'num_successful_orders_future_{time_shot}_month', 'total_amount': f'total_successful_amount_future_{time_shot}_month'}, inplace=True)
        df_snapshot = pd.concat([df_snapshot, df_filter_agg], ignore_index=True) if df_snapshot.shape[0] > 0 else df_filter_agg
        
        # Customers who have  have no successful orders in the filtered data
        df_successful_order = pd.DataFrame(columns=['customer_id', f'total_successful_amount_future_{time_shot}_month', f'num_successful_orders_future_{time_shot}_month'])
        lst_no_orders = [customer_id for customer_id in customers_until_time_snapshot if (customer_id not in lst_customer_filter)]    
        df_successful_order['customer_id'] = lst_no_orders
        df_successful_order.fillna(0, inplace=True)

        df_snapshot = pd.concat([df_snapshot, df_successful_order], ignore_index=True) if df_snapshot.shape[0] > 0 else df_successful_order

        # Add the time_snapshot to the snapshot dataframe
            
    else: # Fill data with NaN
        df_snapshot = pd.DataFrame(columns=['customer_id', f'total_successful_amount_future_{time_shot}_month', f'num_successful_orders_future_{time_shot}_month', 'time_snapshot'])
        df_snapshot['customer_id'] = customers_until_time_snapshot 

    df_snapshot['time_snapshot'] = time_snapshot
    return df_snapshot

In [71]:
# Test the function
time_snapshot = datetime.datetime(2011, 1, 1)
snapshot_future_month_begin(df, time_snapshot, time_shot=1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 4 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   customer_id                             948 non-null    float64       
 1   total_successful_amount_future_1_month  948 non-null    object        
 2   num_successful_orders_future_1_month    948 non-null    object        
 3   time_snapshot                           948 non-null    datetime64[us]
dtypes: datetime64[us](1), float64(1), object(2)
memory usage: 29.8+ KB


In [43]:
# Function to that have snapshot_time as input and return the snapshot of the data from the past and the future
## Input
##      df: the original data
##      snapshot_time: the time of the snapshot
##      past_time_shot: the number of time_spots of the snapshot in the past (1 month ago, 2 months ago, 3 months ago, etc.)
##      future_time_shot: the number of time_spots of the snapshot in the future (1 month later, 2 months later, 3 months later, etc.)
## Output
##      df_snapshot: the snapshot of the data at the given time in the past and the future. The duration of the snapshot is one month
##                   The snapshot includes the total amount of successful orders and the number of successful orders for each customer


def snap_shot_past_future(df: pd.DataFrame, snapshot_time: datetime.datetime, past_time_shot: int = 2, future_time_shot: int = 2) -> pd.DataFrame:
    
    # Get time snapshots
    time_snapshots = df['InvoiceDate'].unique()
    time_min = time_snapshots.min().date()
    time_max = time_snapshots.max().date()
    time_min_snapshot = pd.to_datetime(time_min.replace(day=1))
    time_max_snapshot = pd.to_datetime(time_max.replace(day=1)) + relativedelta(months=1)

    # Get the snapshot of the data in the past at the snapshot_time
    df_past= snapshot_past_month_begin(df, snapshot_time, time_shot=1) # Get the snapshot one month ago

    # Loop to get the snapshots of the data in the past at the snapshot_time
    for i in range(2, past_time_shot+1):
        df_past_temp = snapshot_past_month_begin(df, snapshot_time, time_shot=i) # Get the snapshot i months ago
        df_past_temp.drop(columns=['time_snapshot'], inplace=True) # Drop the time_snapshot column
        #df_past_temp.rename(columns={'total_successful_amount_past_1_month': f'total_successful_amount_past_{i}_month', 'num_successful_orders_past_1_month': f'num_successful_orders_past_{i}_month'}, inplace=True) # Rename the columns name
        df_past = pd.merge(df_past_temp, df_past,  on='customer_id', how='right') # Merge the past snapshots on the customer_id

       

    # Loop to get the snapshots of the data in the future at the snapshot_time
    for i in range(1, future_time_shot+1):
        df_future_temp = snapshot_future_month_begin(df, snapshot_time,  time_shot=i) # Get the snapshot i months later
        df_future_temp.drop(columns=['time_snapshot'], inplace=True) # Drop the time_snapshot column   
        df_past = pd.merge(df_past, df_future_temp,  on='customer_id', how='left') # Merge the future snapshots on the customer_id

    
    # Reorder the columns snapshot_time to the end of the dataframe
    col = df_past.pop('time_snapshot')
    df_past['time_snapshot'] = col
        
    return df_past

In [44]:
# Test the function
snap_shot_past_future(df, time_snapshot, past_time_shot=3, future_time_shot=3)

,customer_id,total_successful_amount_past_3_month,num_successful_orders_past_3_month,total_successful_amount_past_2_month,num_successful_orders_past_2_month,total_successful_amount_past_1_month,num_successful_orders_past_1_month,total_successful_amount_future_1_month,num_successful_orders_future_1_month,total_successful_amount_future_2_month,num_successful_orders_future_2_month,total_successful_amount_future_3_month,num_successful_orders_future_3_month,time_snapshot
0,12347.0,NaN,NaN,NaN,NaN,711.79,31,475.39,29,0,0,0,0,2011-01-01
1,12348.0,NaN,NaN,NaN,NaN,892.80,17,227.44,6,0,0,0,0,2011-01-01
2,12370.0,NaN,NaN,NaN,NaN,1868.02,91,0,0,0,0,938.39,48,2011-01-01
3,12377.0,NaN,NaN,NaN,NaN,1001.52,43,626.6,34,0,0,0,0,2011-01-01
4,12383.0,NaN,NaN,NaN,NaN,600.72,37,639.91,32,0,0,357.07,15,2011-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,18245.0,NaN,NaN,NaN,NaN,365.73,27,439.18,28,-59.5,2,0,0,2011-01-01
944,18256.0,NaN,NaN,NaN,NaN,-50.10,4,0,0,0,0,0,0,2011-01-01
945,18259.0,NaN,NaN,NaN,NaN,376.30,7,0,0,0,0,0,0,2011-01-01
946,18260.0,NaN,NaN,NaN,NaN,230.70,11,538.82,29,0,0,548.75,35,2011-01-01


In [45]:
def snap_shot_all_past_future(df: pd.DataFrame, time_shot=1, past_time_shot = 2, future_time_shot = 2) -> pd.DataFrame:
    
    # Get time snapshots
    time_snapshots = df['InvoiceDate'].unique()
    time_min = time_snapshots.min().date()
    time_max = time_snapshots.max().date()

    # Create the beginning and ending time of the snapshots
    time_begin = pd.to_datetime(time_min.replace(day=1) + relativedelta(months=time_shot))
    time_end = pd.to_datetime(time_max.replace(day=1)+ relativedelta(months=time_shot))

    # Get the first days of each month between the beginning and ending time 
    first_days = get_first_days_of_months(time_begin, time_end)

    # Create a new Dataframe to store the snapshot of the data  
    df_snapshot_all = pd.DataFrame(columns=['customer_id', 'total_successful_amount_past_1_month', 'num_successful_orders_past_1_month', 'time_snapshot'])

    # Get the snapshot of the data at each time in the past
    for time_snapshot in first_days:
        df_snapshot = snap_shot_past_future(df, time_snapshot, past_time_shot=past_time_shot, future_time_shot=future_time_shot)
        df_snapshot_all = pd.concat([df_snapshot_all, df_snapshot], ignore_index=True) if df_snapshot_all.shape[0] > 0 else df_snapshot
    
    return df_snapshot_all

In [47]:
snap_shot_all_past_future(df, past_time_shot=3, future_time_shot=1)

,customer_id,total_successful_amount_past_3_month,num_successful_orders_past_3_month,total_successful_amount_past_2_month,num_successful_orders_past_2_month,total_successful_amount_past_1_month,num_successful_orders_past_1_month,total_successful_amount_future_1_month,num_successful_orders_future_1_month,time_snapshot
0,12347.0,NaN,NaN,NaN,NaN,711.79,31,475.39,29,2011-01-01
1,12348.0,NaN,NaN,NaN,NaN,892.8,17,227.44,6,2011-01-01
2,12370.0,NaN,NaN,NaN,NaN,1868.02,91,0,0,2011-01-01
3,12377.0,NaN,NaN,NaN,NaN,1001.52,43,626.6,34,2011-01-01
4,12383.0,NaN,NaN,NaN,NaN,600.72,37,639.91,32,2011-01-01
...,...,...,...,...,...,...,...,...,...,...
37431,14349.0,NaN,NaN,133.5,31,0,0,NaN,NaN,2012-01-01
37432,18058.0,NaN,NaN,170.16,3,0,0,NaN,NaN,2012-01-01
37433,12953.0,NaN,NaN,329.85,17,0,0,NaN,NaN,2012-01-01
37434,12966.0,NaN,NaN,160.18,10,0,0,NaN,NaN,2012-01-01


In [48]:
def data_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    return snap_shot_all_past_future(df, past_time_shot=4, future_time_shot=2)

In [49]:
df_feature_mart = data_pipeline(df)

In [50]:
output_file = 'd:/code/data/customer_behavior_eom_snapshot.csv'
df_feature_mart.to_csv(output_file, index=False)

In [64]:
df_feature_mart[df_feature_mart.num_successful_orders_past_1_month==0]

,customer_id,total_successful_amount_past_4_month,num_successful_orders_past_4_month,total_successful_amount_past_3_month,num_successful_orders_past_3_month,total_successful_amount_past_2_month,num_successful_orders_past_2_month,total_successful_amount_past_1_month,num_successful_orders_past_1_month,total_successful_amount_future_1_month,num_successful_orders_future_1_month,total_successful_amount_future_2_month,num_successful_orders_future_2_month,time_snapshot
1731,17850.0,NaN,NaN,NaN,NaN,5391.21,297,0,0,-102.58,15,0,0,2011-02-01
1732,13748.0,NaN,NaN,NaN,NaN,204.0,1,0,0,0,0,0,0,2011-02-01
1733,17809.0,NaN,NaN,NaN,NaN,1251.5,15,0,0,610.2,2,0,0,2011-02-01
1734,16098.0,NaN,NaN,NaN,NaN,430.6,12,0,0,421.8,7,0,0,2011-02-01
1735,18074.0,NaN,NaN,NaN,NaN,489.6,13,0,0,0,0,0,0,2011-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37431,14349.0,NaN,NaN,NaN,NaN,133.5,31,0,0,NaN,NaN,NaN,NaN,2012-01-01
37432,18058.0,NaN,NaN,NaN,NaN,170.16,3,0,0,NaN,NaN,NaN,NaN,2012-01-01
37433,12953.0,NaN,NaN,NaN,NaN,329.85,17,0,0,NaN,NaN,NaN,NaN,2012-01-01
37434,12966.0,NaN,NaN,NaN,NaN,160.18,10,0,0,NaN,NaN,NaN,NaN,2012-01-01


In [65]:
df_feature_mart

,customer_id,total_successful_amount_past_4_month,num_successful_orders_past_4_month,total_successful_amount_past_3_month,num_successful_orders_past_3_month,total_successful_amount_past_2_month,num_successful_orders_past_2_month,total_successful_amount_past_1_month,num_successful_orders_past_1_month,total_successful_amount_future_1_month,num_successful_orders_future_1_month,total_successful_amount_future_2_month,num_successful_orders_future_2_month,time_snapshot
0,12347.0,NaN,NaN,NaN,NaN,NaN,NaN,711.79,31,475.39,29,0,0,2011-01-01
1,12348.0,NaN,NaN,NaN,NaN,NaN,NaN,892.8,17,227.44,6,0,0,2011-01-01
2,12370.0,NaN,NaN,NaN,NaN,NaN,NaN,1868.02,91,0,0,0,0,2011-01-01
3,12377.0,NaN,NaN,NaN,NaN,NaN,NaN,1001.52,43,626.6,34,0,0,2011-01-01
4,12383.0,NaN,NaN,NaN,NaN,NaN,NaN,600.72,37,639.91,32,0,0,2011-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37431,14349.0,NaN,NaN,NaN,NaN,133.5,31,0,0,NaN,NaN,NaN,NaN,2012-01-01
37432,18058.0,NaN,NaN,NaN,NaN,170.16,3,0,0,NaN,NaN,NaN,NaN,2012-01-01
37433,12953.0,NaN,NaN,NaN,NaN,329.85,17,0,0,NaN,NaN,NaN,NaN,2012-01-01
37434,12966.0,NaN,NaN,NaN,NaN,160.18,10,0,0,NaN,NaN,NaN,NaN,2012-01-01
